# Detroit Land Bank Authority (DLBA) Occupancy Model

Geoff Perrin - 11.29.17

- This notebook is an update to the occupancy model I built for the DLBA last summer that predicts the occupancy of every single-family residential home in the city of Detroit. This model has many uses, but the initial use case was for the DLBA to better manage their housing inventory, and do a quicker and more accurate job sorting the DLBA housing stock into the oppropriate pipeline (i.e. demolition, buy-back, public auction, etc.)

In [1]:
#import data libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


np.random.seed(222)
%matplotlib inline

In [2]:
#import machine learning / classification libraries
from sklearn.cross_validation import train_test_split
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier as RFC
import graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
pd.options.display.max_columns = 120
pd.options.display.max_rows = 120

### Load Datasets

In [4]:
data = '../data/OccModel_TrainingInputData_20170117/'

#### DTE data

In [5]:
dte_df = pd.read_csv(data + 'DTE_ActiveAgreementsGasElectric_20160824.txt')

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (15,17,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
dte_df = dte_df[['Ref_ID', 'DS_SER_PRODUCT_TYPE']]

In [7]:
### Deduplicate data
dte_df = dte_df.drop_duplicates()

In [8]:
### long to wide reshape on DS_SER_PRODUCT_TYPE
dte_df['gas_dummy'] = [1 if x=='GAS' else 0 for x in dte_df['DS_SER_PRODUCT_TYPE']]
dte_df['electric_dummy'] = [1 if x=='ELC' else 0 for x in dte_df['DS_SER_PRODUCT_TYPE']]
dte_df = dte_df.groupby(['Ref_ID'], as_index=False)['gas_dummy', 'electric_dummy'].sum()

#### DWSD (Water Department) data

In [9]:
dwsd_df = pd.read_csv(data + 'DWSD_12MoUsage_20160815.csv')

In [10]:
dwsd_df = dwsd_df[['Ref_ID', 'Street_ID', 'Add_Geocode', 'ZIP', 
                   u'TOTALACCOUNTBALANCE', u'DECEMBER', u'NOVEMBER',
                   u'OCTOBER', u'SEPTEMBER', u'AUGUST', u'JULY', 
                   u'JUNE', u'MAY', u'APRIL',u'MARCH', u'FEBRUARY', 
                   u'JANUARY']]

In [11]:
dwsd_df['year_total'] = dwsd_df.DECEMBER + dwsd_df.NOVEMBER + dwsd_df.OCTOBER + dwsd_df.SEPTEMBER +\
                        dwsd_df.AUGUST + dwsd_df.JULY +dwsd_df.JUNE + dwsd_df.MAY + dwsd_df.APRIL +\
                        dwsd_df.MARCH + dwsd_df.FEBRUARY + dwsd_df.JANUARY

In [12]:
### sum water amounts of duplicate properties 
dwsd_df = dwsd_df.groupby(['Ref_ID'], as_index=False)[ u'TOTALACCOUNTBALANCE',
       u'DECEMBER', u'NOVEMBER', u'OCTOBER', u'SEPTEMBER', u'AUGUST', u'JULY',
       u'JUNE', u'MAY', u'APRIL', u'MARCH', u'FEBRUARY', u'JANUARY',
       u'year_total'].sum()

#### USPS / DFD (Fire Department) data

In [13]:
usps_df = pd.read_csv(data + 'Fire_USPS_Data_8_18_16.csv')

In [14]:
usps_df.columns = [u'Account Name', u'Ref_ID', u'Property Class', u'USPS: Last Update',
       u'USPS: Vacancy Indicated', u'DFD Fire: Occurrence Date',
       u'DFD Fire: Structure Status']

In [15]:
### subset to residential data
usps_df = usps_df[usps_df['Property Class'] == 'Residential-Structure']

In [16]:
### create DFD fire dummy
usps_df['fire_occured_dummy'] = [0 if pd.isnull(x) else 1 for x in usps_df['DFD Fire: Occurrence Date']]

In [17]:
### create USPS occupied dummy
usps_df['usps_occupied_dummy'] = [1 if x == "No" else 0 for x in usps_df['USPS: Vacancy Indicated']]

In [18]:
### Just keep variables we need
usps_df = usps_df[[u'Ref_ID', u'fire_occured_dummy',
                   u'usps_occupied_dummy']]

#### QVF (voter registration) data

In [19]:
vote_df = pd.read_csv(data + 'QVF_Detroit20160816.csv')

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0,2,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
vote_df = vote_df[['Ref_ID']]
vote_df['vote_dummy'] = 1

In [21]:
vote_df = vote_df.drop_duplicates()

#### Blext data (Motor City Mapping data)
- https://motorcitymapping.org/#t=overview&s=detroit&f=all

In [22]:
mcm_df = pd.read_csv(data + 'Recent_Blexts_8_18_16_v2.csv')

In [23]:
mcm_df.columns = [u'Account Name', u'Ref_ID', u'MCM Condition', u'MCM Structure',
       u'Property Class', u'MCM Occupancy', u'MCM Blext Created At']

In [24]:
### subset to residential data
mcm_df = mcm_df[mcm_df['Property Class'] == 'Residential-Structure']

In [25]:
### subset to non-lot properties
mcm_df = mcm_df[mcm_df['MCM Structure'] == 'Yes']

#### Note from Alice Hargreaves:

NOTE: code "maybe" or "partial" occupied houses as occupied... Ive googled a lot of the "maybes" and theyre nearly all structually sound --- i.e., someone could occupy it, but someone might not be sleeping there every night - GP line of thinking about 3k rows are "maybe" or "partial" 

(R-code below):

In [26]:
def occupancy_binary(x):
    if (x == "Occupied") | (x == "Unoccupied"):
        return x
    else:
        return "Occupied"

In [27]:
mcm_df['MCM Occupancy'] = mcm_df['MCM Occupancy'].apply(occupancy_binary)

### Merge Datasets together

In [28]:
# merge mcm data with usps / dfd data
merged_train = mcm_df.merge(usps_df, how='left', on='Ref_ID')

In [29]:
#merge on voter (qvf) data
merged_train2 = merged_train.merge(vote_df, how='left', on='Ref_ID')

In [30]:
#fill in NaNs for vote_dummy
merged_train2['vote_dummy'] = [0 if pd.isnull(x) else 1 for x in merged_train2['vote_dummy']]

In [31]:
#merge on dwsd data
merged_train3 = merged_train2.merge(dwsd_df, how='left', on='Ref_ID')
#fill in 0s for properties w/ no water
merged_train3.fillna(value=0,inplace=True)

In [32]:
#merge on dte data
merged_train_final = merged_train3.merge(dte_df, how='left', on='Ref_ID')
merged_train_final.fillna(value=0,inplace=True)

In [33]:
merged_train_final.head()

,Account Name,Ref_ID,MCM Condition,MCM Structure,Property Class,MCM Occupancy,MCM Blext Created At,fire_occured_dummy,usps_occupied_dummy,vote_dummy,TOTALACCOUNTBALANCE,DECEMBER,NOVEMBER,OCTOBER,SEPTEMBER,AUGUST,JULY,JUNE,MAY,APRIL,MARCH,FEBRUARY,JANUARY,year_total,gas_dummy,electric_dummy
0,13776 St Louis,13011231.,Good,Yes,Residential-Structure,Occupied,8/16/2016 14:52,0,1,0,129.35,200.0,8000.0,8100.0,200.0,200.0,200.0,200.0,100.0,200.0,100.0,200.0,100.0,17800.0,1.0,1.0
1,13782 St Louis,13011232.,Good,Yes,Residential-Structure,Occupied,8/16/2016 14:52,0,1,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13790 St Louis,13011233.,Good,Yes,Residential-Structure,Occupied,8/16/2016 14:51,0,1,0,27.97,900.0,900.0,800.0,800.0,1100.0,900.0,900.0,800.0,800.0,800.0,800.0,1100.0,10600.0,1.0,1.0
3,13777 St Louis,13011536.,Fair,Yes,Residential-Structure,Unoccupied,8/16/2016 14:50,0,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,13761 St Louis,13011538.,Good,Yes,Residential-Structure,Occupied,8/16/2016 14:49,0,1,0,13.87,400.0,300.0,300.0,300.0,300.0,300.0,300.0,200.0,300.0,200.0,300.0,300.0,3500.0,1.0,1.0
